In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Feature engineering - results analysis

In [45]:
import wandb
import pandas as pd

import plotly.io as pio
pio.renderers.default = 'iframe'

In [46]:
FEATURE_NAMES = [
    'intensity_feature',
    'coordinates_feature',
    'gradient_intensity_feature',
    'texture_contrast_feature',
    'texture_correlation_feature',
    'texture_dissimilarity_feature',
    # 'texture',
    't2_features',
    'edge_feature'
]

## Loading data from W&B

In [47]:
api = wandb.Api()
entity, project = "alicja-krzeminska-sciga-dl", "mialab"

In [48]:
first_valid_run_id = "g85vwre9"
run = api.run(f"{entity}/{project}/{first_valid_run_id}")
first_valid_run_timestamp = run.summary["_timestamp"]

In [49]:
runs = api.runs(entity + "/" + project)

run_dict_list = []

for run in runs:
    if run.summary["_timestamp"] >= first_valid_run_timestamp:
        summary = run.summary._json_dict
        config = {k: v for k, v in run.config.items() if not k.startswith("_")}
        
        run_dict_list.append({
            "run_id": run.id,
            "run_name": run.name,
            **summary,
            **config
        })
        
runs_df = pd.DataFrame(run_dict_list)

In [50]:
metric_columns = [c for c in runs_df.columns if "_DICE" in c]
interesting_columns = metric_columns + FEATURE_NAMES + ["n_estimators", "texture_window"]

runs_df[interesting_columns].head()

,MEAN_DICE,MEAN_DICE_Amygdala,MEAN_DICE_GreyMatter,MEAN_DICE_Hippocampus,MEAN_DICE_Thalamus,MEAN_DICE_WhiteMatter,STD_DICE_Amygdala,STD_DICE_GreyMatter,STD_DICE_Hippocampus,STD_DICE_Thalamus,...,intensity_feature,coordinates_feature,gradient_intensity_feature,texture_contrast_feature,texture_correlation_feature,texture_dissimilarity_feature,t2_features,edge_feature,n_estimators,texture_window
0,0.393097,0.0,0.649125,0.128059,0.417522,0.770781,0.0,0.017915,0.009357,0.019323,...,True,True,True,True,True,True,True,True,100,10
1,0.353196,0.0,0.643616,0.125801,0.218295,0.778271,0.0,0.017772,0.009161,0.014559,...,True,True,True,True,True,True,True,False,100,10
2,0.307565,0.0,0.584127,0.117544,0.129015,0.707138,0.0,0.038955,0.027173,0.103484,...,True,True,True,True,True,True,False,True,100,10
3,0.298819,0.0,0.570537,0.087176,0.132333,0.704048,0.0,0.038195,0.016760,0.107087,...,True,True,True,True,True,True,False,False,100,10
4,0.353528,0.0,0.647737,0.125849,0.218995,0.775057,0.0,0.018799,0.009740,0.014930,...,True,True,True,False,False,False,True,True,100,10


In [51]:
runs_df = runs_df.drop_duplicates(subset=FEATURE_NAMES + ["n_estimators", "texture_window"], keep='first')

In [52]:
runs_df[["n_estimators", "texture_window"]].value_counts()

n_estimators  texture_window
10            5                 62
100           5                 62
              10                62
Name: count, dtype: int64

## Reslults analysis

In [53]:
from plot_results import plot_dice_scores

### A larger amount of features improves the segmentation performance

In [54]:
def count_features(row):
    features_count = 0
    for f in FEATURE_NAMES:
        if f != "t2_features":
            features_count += row[f]
    if row["t2_features"]:
        features_count *= 2
    return features_count

runs_df["features_number"] = runs_df.apply(lambda row: count_features(row), axis=1)

In [55]:
runs_df.columns

Index(['run_id', 'run_name', 'DICE score', 'Features importance', 'MEAN_DICE',
       'MEAN_DICE_Amygdala', 'MEAN_DICE_GreyMatter', 'MEAN_DICE_Hippocampus',
       'MEAN_DICE_Thalamus', 'MEAN_DICE_WhiteMatter', 'STD_DICE_Amygdala',
       'STD_DICE_GreyMatter', 'STD_DICE_Hippocampus', 'STD_DICE_Thalamus',
       'STD_DICE_WhiteMatter', '_runtime', '_step', '_timestamp', '_wandb',
       'coordinates_feature', 'edge_feature', 'gradient_intensity_feature',
       'intensity_feature', 'load_features', 'load_pre', 'normalization_pre',
       'overwrite', 'registration_pre', 'save_features', 'save_pre',
       'skullstrip_pre', 't2_features', 'texture_contrast_feature',
       'texture_correlation_feature', 'texture_dissimilarity_feature',
       'texture_window', 'training', 'n_jobs', 'verbose', 'bootstrap',
       'ccp_alpha', 'criterion', 'max_depth', 'oob_score', 'warm_start',
       'max_samples', 'class_weight', 'max_features', 'n_estimators',
       'random_state', 'max_leaf_nodes', 

In [58]:
plot_dice_scores(runs_df, n_estimators=100, texture_window=10, metric_columns=metric_columns)

In [ ]:
plot_dice_scores(runs_df, n_estimators=100, texture_window=5)

In [ ]:
plot_dice_scores(runs_df, n_estimators=10, texture_window=5)

In general, we can see increasing trend of metrics values with respect to the number of features. However, after analysis of DICE score for each class we can see that the biggest impact the number of features has on GreyMatter and WhiteMatter, that have quite big DICE scores, so their contribution to mean DICE is bigger than for other classes.

Additionally, here we consider only the number of the features, but the combinations are different, and also the number of samples is different (e.g. only one sample for all the features activated).

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

runs_df['n_estimators_and_texture_window'] = (
    runs_df['n_estimators'].astype(str) + '&' + runs_df['texture_window'].astype(str)
)

grouped_df = runs_df.groupby(FEATURE_NAMES + ['n_estimators_and_texture_window'])['MEAN_DICE'].mean().reset_index()

pivot_df = grouped_df.pivot(
    index=FEATURE_NAMES,
    columns='n_estimators_and_texture_window',
    values='MEAN_DICE'
).reset_index()

pivot_df = pivot_df.sort_values("100&5").reset_index()


fig = go.Figure()

n_estimators_columns = [col for col in pivot_df.columns if col not in FEATURE_NAMES + ["index"]]

for col in n_estimators_columns:
    fig.add_trace(
        go.Scatter(
            x=pivot_df.index,
            y=pivot_df[col],
            mode='markers',
            name=col
        )
    )

fig.update_layout(
    title="MEAN_Dice for each n_estimators_and_texture_window",
    xaxis_title="Feature Combination",
    yaxis_title="MEAN_Dice",
    legend_title="n_estimators_and_texture_window",
)

fig.show()

### Feature importance analysis

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread(f"mia-result/g85vwre9_feature_importances.png")
plt.figure(figsize=(10, 10)) 
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

From the analysis of the feature importances we can see that the intensity of T1w and T2w are the most important for the model.